In [1]:
!pwd

/home/grad16/sakumar/sparkNMSU


In [ ]:
# import matplotlib.pyplot as plt
import numpy as np

import os
os.environ["JAVA_HOME"] = "/dbr1/hctl/spark_essentials/jdk1.8.0_251/"
os.environ["SPARK_HOME"] = "/usr/local/spark-2.4.0-bin-hadoop2.7/"

import findspark
findspark.init()
from pyspark_iforest.ml.iforest import *
from decimal import Decimal
from pyspark.ml.linalg import Vectors
import tempfile
import pandas as pd
from pyspark.sql.functions import to_timestamp,to_date
from datetime import datetime
import time as realtime
from pyspark.sql import SQLContext,SparkSession
import shutil

%matplotlib inline
# %matplotlib notebook

spark = SparkSession.builder.master("local[*]").appName("IForestExample").config("spark.jars", "/dbr1/hctl/spark_essentials/mysql-connector-java-5.1.49.jar,/dbr1/hctl/spark_essentials/spark-iforest-2.4.0.jar").getOrCreate()

In [ ]:
# use ggplot style for more sophisticated visuals
plt.style.use('ggplot')

def live_plotter(x_vec,y1_data,line1,identifier='',pause_time=0.1):
    if line1==[]:
        # this is the call to matplotlib that allows dynamic plotting
        #plt.ion()
        fig = plt.figure(figsize=(13,6))
        ax = fig.add_subplot(111)
        # create a variable for the line so we can later update it
        line1, = ax.plot(x_vec,y1_data,'-',alpha=0.8)        
        #update plot label/title
        plt.ylabel('Y Label')
        plt.title('Title:  {}'.format(identifier))
        #plt.show()
    
    # after the figure, axis, and line are created, we only need to update the y-data
    line1.set_ydata(y1_data)   
#     line1.set_xdata(x_vec)
       
    # adjust limits if new data goes beyond bounds
    if np.min(y1_data)<=line1.axes.get_ylim()[0] or np.max(y1_data)>=line1.axes.get_ylim()[1]:
        plt.ylim([np.min(y1_data)-np.std(y1_data),np.max(y1_data)+np.std(y1_data)])
#     plt.xlim(x_vec[-2],x_vec[-1])
#     if np.min(x1_data)<=line1.axes.get_xlim()[0] or np.max(x1_data)>=line1.axes.get_xlim()[1]:
#          plt.xlim([np.min(x1_data)-np.std(x1_data),np.max(y1_data)+np.std(y1_data)])
    # this pauses the data so the figure/axis can catch up - the amount of pause can be altered above
    #plt.pause(pause_time)
    
    # return line so we can update it again in the next iteration
    return line1


def live_plotters(x_vec,y1_data,y2_data,y3_data,y4_data,y5_data,y6_data,line1,line2,line3,line4,line5,line6,anomaly_date,pred_list,identifier='',pause_time=0.1):
    if line1==[]:
       
        # this is the call to matplotlib that allows dynamic plotting
        #plt.ion()
        
        fig = plt.figure(figsize=(13,6))
        ax = fig.add_subplot(111)
        i=0
        # create a variable for the line so we can later update it
        #line1, = ax.plot(x_vec,y1_data,'-',alpha=0.8) 
        line2, = ax.plot(x_vec,y2_data,'-',alpha=0.8)
        line3, = ax.plot(x_vec,y3_data,'-',alpha=0.8)
        line4, = ax.plot(x_vec,y4_data,'-',alpha=0.8)
        line5, = ax.plot(x_vec,y5_data,'-',alpha=0.8)
        line6, = ax.plot(x_vec,y6_data,'-',alpha=0.8)
#         line7, = ax.plot(x_vec,y7_data,'-',alpha=0.8)
#         line8, = ax.plot(x_vec,y8_data,'-',alpha=0.8)
 
        #line1.set_label('Score(testing)')
        line2.set_label('glp')
        line3.set_label('grp')
        line4.set_label('sub_metering_1')
        line5.set_label('sub_metering_2')
        line6.set_label('sub_metering_3')
#         line7.set_label('Voltage')
#         line8.set_label('Global Index')
        
        ax.legend()
#         ax.set_xlim(left=max(datetime(2006, 12, 21, 4, 6),x_vec[i]), right=x_vec[i+5])
        
#         ax.scatter(anomaly_date,pred_list)
        #update plot label/title
        plt.ylabel('Y Label')
        plt.title('Title:{}'.format(identifier))
        plt.draw()
#         plt.show()
    # after the figure, axis, and line are created, we only need to update the y-data
    
    #line1.set_ydata(y1_data)
    line2.set_ydata(y2_data)
    line3.set_ydata(y3_data)
    line4.set_ydata(y4_data)
    line5.set_ydata(y5_data)
    line6.set_ydata(y6_data)
#     line7.set_ydata(y7_data)
#     line8.set_ydata(y8_data)
    
    # adjust limits if new data goes beyond bounds
    plt.gca().set_xlim(left=x_vec[0])
    #if np.min(y1_data)<=line1.axes.get_ylim()[0] or np.max(y1_data)>=line1.axes.get_ylim()[1]:
    #    plt.ylim([np.min(y1_data)-np.std(y1_data),np.max(y1_data)+np.std(y1_data)])
    if np.min(y2_data)<=line2.axes.get_ylim()[0] or np.max(y2_data)>=line2.axes.get_ylim()[1]:
        plt.ylim([np.min(y2_data)-np.std(y1_data),np.max(y2_data)+np.std(y2_data)])   
    if np.min(y3_data)<=line3.axes.get_ylim()[0] or np.max(y3_data)>=line3.axes.get_ylim()[1]:
        plt.ylim([np.min(y3_data)-np.std(y1_data),np.max(y3_data)+np.std(y3_data)])
    if np.min(y4_data)<=line4.axes.get_ylim()[0] or np.max(y4_data)>=line4.axes.get_ylim()[1]:
        plt.ylim([np.min(y4_data)-np.std(y1_data),np.max(y4_data)+np.std(y4_data)])
    if np.min(y5_data)<=line5.axes.get_ylim()[0] or np.max(y5_data)>=line5.axes.get_ylim()[1]:
        plt.ylim([np.min(y5_data)-np.std(y1_data),np.max(y5_data)+np.std(y5_data)])   
    if np.min(y6_data)<=line6.axes.get_ylim()[0] or np.max(y6_data)>=line6.axes.get_ylim()[1]:
        plt.ylim([np.min(y6_data)-np.std(y1_data),np.max(y6_data)+np.std(y6_data)])
    # this pauses the data so the figure/axis can catch up - the amount of pause can be altered above
    #plt.pause(pause_time)
    
    # return line so we can update it again in the next iteration
    return line2,line3,line4,line5,line6



In [ ]:
import plotly.graph_objects as go

fig = go.Figure(
    data=[go.Scatter(x=[0, 1], y=[0, 1])],
    layout=go.Layout(
        xaxis=dict(range=[0, 5], autorange=False),
        yaxis=dict(range=[0, 5], autorange=False),
        title="Start Title",
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None])])]
    ),
    frames=[go.Frame(data=[go.Scatter(x=[1, 2], y=[1, 2])]),
            go.Frame(data=[go.Scatter(x=[1, 4], y=[1, 4])]),
            go.Frame(data=[go.Scatter(x=[3, 4], y=[3, 4])],
                     layout=go.Layout(title_text="End Title"))]
)

fig.show()

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objects as go

def anomaly_map(anomaly_date,glp,sub1,sub2,sub3,size):
    glp_data_anomaly,sub1_data_anomaly,sub2_data_anomaly,sub3_data_anomaly=[[] for _ in range(4)]
    for i in anomaly_date:
        for j in range(size):
            if(i==x_vec[j]):
                glp_data_anomaly.append(glp[j])
                sub1_data_anomaly.append(sub1[j]) 
                sub2_data_anomaly.append(sub2[j]) 
                sub3_data_anomaly.append(sub3[j])
                
    return glp_data_anomaly,sub1_data_anomaly,sub2_data_anomaly,sub3_data_anomaly

def goScatter(showlegend,y):
     return go.Scatter(name="Anomaly",
                               showlegend=showlegend,
                               x=anomaly_date,
                               y=y,
                               hovertext = ["Score: "+str(p) for p in pred_list],
                       #       text = ["Score: "+str(p) for p in pred_list],
                       #       hoverinfo = 'text',
                               mode='markers',
                               marker=dict(color="orange",
                                           size=11,
                                           line=dict(
                                               color="red",
                                               width=2)))

def live_plotters1(x_vec,anomalyScores_list,y2_data,y3_data,y4_data,y5_data,y6_data,anomaly_date,pred_list,pause_time=0.1):

    plot_data2=go.Scatter(name='glp',showlegend=True,x=x_vec, y=y2_data,line=dict(color='firebrick', width=4))
    plot_data4=go.Scatter(name='kitchen',showlegend=True,x=x_vec, y=y4_data)
    plot_data5=go.Scatter(name='laundry',showlegend=True,x=x_vec, y=y5_data)
    plot_data6=go.Scatter(name='AC',showlegend=True,x=x_vec, y=y6_data)
    
    y2_data_anomaly,y4_data_anomaly,y5_data_anomaly,y6_data_anomaly = anomaly_map(anomaly_date,y2_data,y4_data,y5_data,y6_data,len(x_vec))       
    
    anomalies_map2 = goScatter(showlegend=True,y=y2_data_anomaly)
    anomalies_map4 = goScatter(showlegend=False,y=y4_data_anomaly)
    anomalies_map5 = goScatter(showlegend=False,y=y5_data_anomaly)
    anomalies_map6 = goScatter(showlegend=False,y=y6_data_anomaly)
    
    fig=go.Figure(data = [plot_data2,
                        plot_data4,plot_data5,plot_data6,
                        anomalies_map2
                        ,anomalies_map4,anomalies_map5,anomalies_map6
                       ])

    
    fig.update_layout(title='Housing Power Consumption',
                   xaxis_title='Date',
                   yaxis_title='Power',
                   plot_bgcolor='white')
#     print(anomalyScores_list)
#     print(pred_list)
    return fig

In [ ]:
def read_db(spark,current): 
    # spark = SparkSession.builder.master("local[*]")             .appName("IForestExample")             .config("spark.jars", "/dbr1/hctl/spark_essentials/mysql-connector-java-5.1.49.jar,/dbr1/hctl/spark_essentials/spark-iforest-2.4.0.jar").getOrCreate()
    # spark = SparkSession.builder.master("local[*]").config("spark.jars", "/dbr1/hctl/spark_essentials/mysql-connector-java-5.1.49.jar,/dbr1/hctl/spark_essentials/spark-iforest-2.4.0.jar").getOrCreate()
   
    sqlContext = SQLContext(spark.sparkContext)
    #print("Spark Version - ", spark.version)
    hostname = "dbclass.cs.nmsu.edu" 
    dbname = "nmsu_power"
    jdbcPort = 3306
    username = "sakumar"
    password = "dbnmsu123"
    jdbc_url = "jdbc:mysql://{0}:{1}/{2}?user={3}&password={4}".format(hostname,jdbcPort, dbname,username,password)
    query = """
    ( SELECT * FROM power where ID >"""+str(current)+""" and ID <="""+str(current+50)+""" ) whyalias
    """
    #print(query)
    df_origin = sqlContext.read.format('jdbc').options(driver='com.mysql.jdbc.Driver',url=jdbc_url, dbtable=query ).load()
    return df_origin

In [ ]:
def df_to_list(iterate_value):
    iterate_list = []
    for iv in iterate_value:
      row_list=[]
      for i in iv:
       if type(i) == Decimal:
        row_list.append(float(i))
       elif type(i) == float:
        row_list.append(float(i))
       elif type(i) == datetime:
        row_list.append(i.time())
       else:
        row_list.append(i)
      iterate_list.append(row_list)
    return iterate_list


def Decimal_append(g):
    a=[]
    for row in g:
        row_data = []
        for data in row:
            if type(data) == Decimal:
                row_data.append(float(data))
            elif type(data) == float:
                row_data.append(float(data))
        a.append(row_data)
    return a

def iforest_spark(data):
##########################

# Spark-iForest  

##########################

# NOTE: features need to be dense vectors for the model input

    df = spark.createDataFrame(data, ["features"])

    # Init an IForest Object
    iforest = IForest(contamination=0.1,maxDepth=4)
    #print("init iforest")
    # Fit on a given data frame
    model = iforest.fit(df)
    #print("fit iforest")
    # Check if the model has summary or not, the newly trained model has the summary info
    model.hasSummary

    # Show model summary
    summary = model.summary

    # Show the number of anomalies
    summary.numAnomalies

    # Predict for a new data frame based on the fitted model
    transformed = model.transform(df)

    # Collect spark data frame into local df
    rows = transformed.collect()

#     temp_path = tempfile.mkdtemp()
    
    temp_path = '/home/grad16/sakumar/sparkNMSU/tmp/'
  
    shutil.rmtree(temp_path)
    os.mkdir(temp_path)
    
    iforest_path = temp_path + "/iforest"
    
    # Save the iforest estimator into the path
    iforest.save(iforest_path)

    # Load iforest estimator from a path
    loaded_iforest = IForest.load(iforest_path)

    model_path = temp_path + "/iforest_model"

    # Save the fitted model into the model path
    model.save(model_path)

    # Load a fitted model from a model path
    loaded_model = IForestModel.load(model_path)

    # The loaded model has no summary info
    loaded_model.hasSummary

    # Use the loaded model to predict a new data frame
    new_df=loaded_model.transform(df)
#     new_df.show()
    return new_df

In [ ]:
def iforest_run(current):
    
    df_origin = read_db(spark,current)
#     df_origin.show() 
    df_origin = df_origin.na.drop()
#     print((df_origin.count(), len(df_origin.columns)))
###### Data Conversion ########

    date=df_origin.select(to_date(df_origin['Date'], 'dd/MM/yyyy').alias('date')).collect()
    df_time=df_origin.select(to_timestamp(df_origin['Time'], 'yyyy-MM-dd HH:mm:ss').alias('time')).collect()
    date_list= df_to_list(date)
    time_list= df_to_list(df_time)

    date_time_list=[]
    for i in range(len(time_list)):
      date_time_list.append(datetime.combine(date_list[i][0],time_list[i][0]))
    
    columns_to_drop=["Date","Time"]
    df1=df_origin.drop(*columns_to_drop)

    a=[column for column in df1.columns]
    g=df1.select(a).collect()
    size=0
    a=Decimal_append(g)
    
    # a=a[-last:]
    a=[[i / sum(j) for i in j] for j in a]
    data = [(Vectors.dense(i),) for i in a]
    
    new_df = iforest_spark(data)
    
    columns_to_drop=["Date","Time"]
    df1=df_origin.drop(*columns_to_drop)
    
    anomalyScores_list = [row['anomalyScore'] for row in new_df.toLocalIterator()]
    plist= [row['prediction'] for row in new_df.toLocalIterator()]

    anomaly_date=[]
    for i,j in zip(plist,date_time_list):
      if(i==1.0):
        anomaly_date.append(j)

    pred_1=new_df.filter(new_df['prediction']==1.0).select(new_df['anomalyScore']).collect()  
    pred_list= df_to_list(pred_1)    
    pred_list = [item for sublist in pred_list for item in sublist]
    
    glp = [float(row['global_active_power']) for row in df_origin.toLocalIterator()]
    grp = [float(row['global_reactive_power']) for row in df_origin.toLocalIterator()]
    sub1= [row['sub_metering_1'] for row in df_origin.toLocalIterator()]
    sub2= [row['sub_metering_2'] for row in df_origin.toLocalIterator()]
    sub3= [float(row['sub_metering_3']) for row in df_origin.toLocalIterator()]
    vol = [float(row['voltage']) for row in df_origin.toLocalIterator()]
    gi = [float(row['global_intensity']) for row in df_origin.toLocalIterator()]
    
    return(date_time_list,glp, grp,sub1,sub2,sub3,anomalyScores_list,anomaly_date,pred_list,vol,gi)

In [6]:
import numpy as np
from IPython import display
import time
%matplotlib inline


##############################################################################
#current = 9656
current = 1

line1 , line2, line3, line4, line5, line6= [[] for _ in range(6)]

while True:
    x_vec,glp, grp,sub1,sub2,sub3,anomalyScores_list,anomaly_date,pred_list,vol,gi=iforest_run(current) 
    y_vec, y_vec2, y_vec3,y_vec4,y_vec5,y_vec6 = anomalyScores_list,glp, grp,sub1,sub2,sub3
    
#     line1 = live_plotter(x_vec,y_vec,line1)
#     line2,line3,line4,line5,line6 = live_plotters(x_vec,y_vec,y_vec2,y_vec3,y_vec4,y_vec5,y_vec6,line1,line2,line3,line4,line5,line6,anomaly_date,pred_list)
    
    fig = live_plotters1(x_vec,y_vec,y_vec2,y_vec3,y_vec4,y_vec5,y_vec6,anomaly_date,pred_list)
#     ax.set_xlim(left=date_time_list[i], right=date_time_list[i+5])
#    plt.gcf().canvas.draw()
#     display.display(plt.gcf())

    display.display(fig)
    display.clear_output(wait=True)
    current = current + 1

KeyboardInterrupt: 

In [ ]:
import numpy as np
from IPython import display
%matplotlib inline

def goScatter(showlegend,y):
    
     return go.Scatter(name="Anomaly",
                               showlegend=showlegend,
                               x=anomaly_date,
                               y=y,
                               hovertext = ["Score: "+str(p) for p in pred_list],
                       #       text = ["Score: "+str(p) for p in pred_list],
                       #       hoverinfo = 'text',
                               mode='markers',
                               marker=dict(color="orange",
                                           size=11,
                                           line=dict(
                                               color="red",
                                               width=2)))


def anomaly_map(anomaly_date,glp,sub1,sub2,sub3,size):
    glp_data_anomaly,sub1_data_anomaly,sub2_data_anomaly,sub3_data_anomaly=[[] for _ in range(4)]
    for i in anomaly_date:
        for j in range(size):
            if(i==x_vec[j]):
                glp_data_anomaly.append(glp[j])
                sub1_data_anomaly.append(sub1[j]) 
                sub2_data_anomaly.append(sub2[j]) 
                sub3_data_anomaly.append(sub3[j])
                
    return glp_data_anomaly,sub1_data_anomaly,sub2_data_anomaly,sub3_data_anomaly


#current = 9656
current = 19960
max_ID= 19965

frames = []

while current < max_ID:
    x_vec,glp,grp,sub1,sub2,sub3,anomalyScores_list,anomaly_date,pred_list,vol,gi=iforest_run(current) 
    print(current)
    plot_glp = go.Scatter(name='glp',showlegend=True,x=x_vec, y=glp,line=dict(color='firebrick', width=4))

#     plot_sub1 = go.Scatter(name='kitchen',showlegend=True,x=x_vec, y=sub1)
#     plot_sub2 = go.Scatter(name='laundry',showlegend=True,x=x_vec, y=sub2)
#     plot_sub3 = go.Scatter(name='AC',showlegend=True,x=x_vec, y=sub3)
    
#     glp_data_anomaly,sub1_data_anomaly,sub2_data_anomaly,sub3_data_anomaly=[[] for _ in range(4)]
#     glp_data_anomaly,sub1_data_anomaly,sub2_data_anomaly,sub3_data_anomaly = anomaly_map(anomaly_date,glp,sub1,sub2,sub3,len(x_vec))
    
#     anomalies_glp = goScatter(showlegend=True,y=glp_data_anomaly)
#     anomalies_sub1 = goScatter(showlegend=False,y=sub1_data_anomaly)
#     anomalies_sub2 = goScatter(showlegend=False,y=sub2_data_anomaly)
#     anomalies_sub3 = goScatter(showlegend=False,y=sub3_data_anomaly)
    
    current_frame = go.Frame(data = go.Scatter(name='glp',showlegend=True,x=x_vec, y=glp,line=dict(color='firebrick', width=4)))
#                                          ,plot_sub1,plot_sub2,plot_sub3,anomalies_glp,anomalies_sub1,anomalies_sub2,anomalies_sub3
                                        
    frames.append(current_frame)
    current= current+1

fig = go.Figure(layout=go.Layout(
                         updatemenus=[dict(
                                type="buttons",
                                buttons=[dict(label="Play",
                          method="animate",
                          args=[None])])]
    ),
                frames=frames
                )
display.display(fig)
display.clear_output(wait=True)

In [ ]:
import time
import matplotlib.pyplot as plt

# %matplotlib notebook

# plt.rcParams['animation.html'] = 'jshtml'

fig = plt.figure(figsize=(13,6))
ax = fig.add_subplot(111)
ax.set_ylabel('Power Consumption')
ax.set_xlabel('Timestamp')
# ax.legend(handles=['Score'])
# ax.legend()
fig.show()

current = 9656

x_vec,glp, grp,sub1,sub2,sub3,anomalyScores_list,anomaly_date,pred_list,vol,gi=iforest_run(current)

i = 0
j = 0
d = 0

x, y =[] , []

# s1,s2,s3 = [], [] , [] , []

# print(date_time_list[2])

while True:
    x.append(x_vec[i])
    y.append(anomalyScores_list[i])
#     s1.append(sub1[i])
#     s2.append(sub2[i])
#     s3.append(sub3[i])
#     ax.plot(x, glp, color='b',label='Score')
    ax.plot(x,y, color='b')
#     ax.plot(x,s2,color='g')
#     ax.plot(x,s3,color='r')
    
    if(anomaly_date[d] == x_vec[i]):
        ax.scatter(anomaly_date[d],pred_list[d])
        d=d+1
    fig.canvas.draw()
    
    ax.set_xlim(left=max(datetime(2006, 12, 21, 4, 6),x_vec[i]), right=x_vec[i+5])
#     ax.set_xlim(left=max(0, i-50), right=i+50)
    
    time.sleep(1)
    i = i +  1
    j = j + 2
    

plt.close()

In [ ]:
import numpy as np
from IPython import display
import time

size = 50
x_vec = np.linspace(0,1,size+1)[0:-1]
y_vec = np.random.randn(len(x_vec))
line1 = []

while True:
    #iforest
    #read last 100 datapoints
    #run iforest
    #draw
    
    rand_val = np.random.randn(1)
    y_vec[-1] = rand_val
    y_vec = np.append(y_vec[1:],0.0)

#     line1 = live_plotters(x_vec,y_vec,y_vec2,line1,line2)
    line1 = live_plotter(x_vec,y_vec,line1)
    display.display(plt.gcf())
    display.clear_output(wait=True)
    
    time.sleep(0.1)

In [ ]:
import numpy as np
from IPython import display
# from iforest import iforest_spark

size = 50
x_vec = np.linspace(0,1,size+1)[0:-1]
y_vec = np.random.randn(len(x_vec))
line1 = []

x_vec2 = np.linspace(0,1,size+1)[0:-1]
y_vec2 = 4 * np.random.randn(len(x_vec2))
line2 = []

# print(y_vec)
# print(x_vec)


while True:
    #iforest
    #read last 100 datapoints
    #run iforest
    #draw
    
    rand_val = np.random.randn(1)
    y_vec[-1] = rand_val
    y_vec = np.append(y_vec[1:],0.0)
    
    rand_val2 = np.random.randn(1)
    y_vec2[-1] = 4 * rand_val2
    y_vec2 = np.append(y_vec2[1:],0.0)
    
    line1,line2 = live_plotters(x_vec,y_vec,y_vec2,y_vec3,y_vec4,y_vec5, y_vec6,line1,line2,line3,line4,line5,line6,anomaly_date,pred_list)
    
#     line1 = live_plotter(x_vec,y_vec,line1)
    display.display(plt.gcf())
    display.clear_output(wait=True)
    
    time.sleep(0.1)
    

In [ ]:
%matplotlib inline
import time
import pylab as pl
from IPython import display
for i in range(10):
    pl.plot(pl.randn(100))
    display.display(pl.gcf())
    display.clear_output(wait=True)
    time.sleep(1.0)